In [1]:
# 순환신경망
# LSTM -> 가중치, 절편, 순환가중치 셀 4개 순환시키는 신경망
# 여기선 양방향 LSTM
# 앞의 훈련 결과를 다음 타임스탭에 전달
# 역방향을 추가
# 문장의 뒷부분부터 훈련해서 앞으로 가는데 앞부분의 데이터도 잘 훈련할 수 있다.

In [ ]:
# NER(개체명 인식)
# 문장에서 인물, 장소, 날짜 등을 의미하는 단어인지 인식
# 오늘 부산 날씨 알려줘
# 날짜 장소

In [ ]:
# BIO 표기법
# Beginning, Inside, Outside
# 개체명이 시작되는 단어 B
# I => B에 이어지는 개체명
# O => 그 외

# 삼성전자에 다닌다.
# 삼성전자 => 회사라는 개체명 인식
# 삼성(B-회사), 전자(I-회사), 에(O)

In [2]:
# train.txt
# 첫줄 ; 시작되는 원본 문장
# 두번째 줄 $ 시작되는 NER처리된 문장
# 그밑에서부터 토큰번호, 토큰, 품사태그, BIO태그
# 빈 줄

In [51]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np
from seqeval.metrics import f1_score
from sklearn.metrics import classification_report


In [3]:
def read_file(filename):
    sent = []
    with open(filename, 'r', encoding='utf-8') as f :
        lines = f.readlines()
        for idx, l in enumerate(lines):
            # 새로운 문장이 시작될 때 초기화
            if l[0] == ';' and lines[idx + 1][0] == '$':
                this_sent = []
            # 두번째 줄 아무동작 안하고 건너뜀
            elif l[0] == '$' and lines[idx-1][0] == ';':
                continue
            # 마지막 줄(비어있는 줄) sent배열을 채워주기
            elif l[0] == '\n':
                sent.append(this_sent)
            else:
                this_sent.append(tuple(l.split()))
    return sent

In [4]:
corpus = read_file('./data/train.txt')

In [5]:
# BIO 태그만 불러와 데이터셋 생성
sentence = []
tag = []
for t in corpus :
    sen = []
    bio = []
    for w in t:
        sen.append(w[1])
        bio.append(w[3])
    sentence.append(sen)
    tag.append(bio)

In [6]:
len(sentence)

3555

In [7]:
sentence[0]

['한편',
 ',',
 'AFC',
 '챔피언스',
 '리그',
 'E',
 '조',
 '에',
 '속하',
 'ㄴ',
 '포항',
 '역시',
 '대회',
 '8강',
 '진출',
 '이',
 '불투명',
 '하',
 '다',
 '.']

In [8]:
print(tag[0])

['O', 'O', 'O', 'O', 'O', 'B_OG', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [10]:
# 토크나어저

# oov -> 단어사전에 없는 단어를 <OOV>라고 인식
sent_tokenizer = keras.preprocessing.text.Tokenizer(oov_token='OOV')
sent_tokenizer.fit_on_texts(sentence)

# 자동으로 소문자 바뀌는거 false
tag_tokenizer = keras.preprocessing.text.Tokenizer(lower=False)
tag_tokenizer.fit_on_texts(tag)

In [11]:
voca_size = len(sent_tokenizer.word_index) + 1
tag_size = len(tag_tokenizer.word_index) + 1

# 단어 시퀀스
x_train = sent_tokenizer.texts_to_sequences(sentence)
y_train = tag_tokenizer.texts_to_sequences(tag)

In [12]:
# index to word/ index to ner -> (숫자로 된 데이터를 다시 한글로 돌리기 위해 미리 정의)
index_to_word = sent_tokenizer.index_word
index_to_ner = tag_tokenizer.index_word
index_to_ner[0] = 'PAD'

In [13]:
x_train = keras.utils.pad_sequences(x_train, padding='post', maxlen=40)
y_train = keras.utils.pad_sequences(y_train, padding='post', maxlen=40)

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x_train,y_train, test_size=0.2)

In [15]:
# 원 핫 인코딩

y_train = tf.keras.utils.to_categorical(y_train, num_classes=tag_size)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=tag_size)

In [ ]:
# 여기까지가 데이터 전처리

In [19]:
# 모델 Bi-LSTM

model = keras.Sequential()

# mask_zero -> 패딩으로 만들어진 0값을 다음 레이어로 전달하지 않는다
model.add(keras.layers.Embedding(input_dim=voca_size, output_dim=30, input_length=40, mask_zero=True))

# Bidirectional -> 양방향
# dropout -> 레이어 빠져나갈때 드롭아웃
# recurrent_dropout -> 순환할때 드롭아웃
model.add(keras.layers.Bidirectional(keras.layers.LSTM(200, return_sequences=True, dropout=0.5, recurrent_dropout=0.25)))

# TimeDistribution -> 밀집층에서 3차원 데이터 받을 수 있게
model.add(keras.layers.TimeDistributed(keras.layers.Dense(tag_size,activation='softmax')))

In [21]:
# sparse -> 이미 원핫인코딩이 되어있는 인풋데이터면 안 써도 됨
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=128, epochs=10)

Epoch 1/10
23/23 [==============================] - 19s 423ms/step - loss: 1.1757 - accuracy: 0.8359
Epoch 2/10
23/23 [==============================] - 11s 481ms/step - loss: 0.6160 - accuracy: 0.8682
Epoch 3/10
23/23 [==============================] - 11s 481ms/step - loss: 0.5865 - accuracy: 0.8682
Epoch 4/10
23/23 [==============================] - 12s 527ms/step - loss: 0.5581 - accuracy: 0.8683
Epoch 5/10
23/23 [==============================] - 12s 517ms/step - loss: 0.5195 - accuracy: 0.8677
Epoch 6/10
23/23 [==============================] - 12s 513ms/step - loss: 0.4423 - accuracy: 0.8754
Epoch 7/10
23/23 [==============================] - 12s 520ms/step - loss: 0.3709 - accuracy: 0.8876
Epoch 8/10
23/23 [==============================] - 13s 545ms/step - loss: 0.3187 - accuracy: 0.8960
Epoch 9/10
23/23 [==============================] - 13s 584ms/step - loss: 0.2855 - accuracy: 0.9025
Epoch 10/10
23/23 [==============================] - 14s 599ms/step - loss: 0.2634 - accura

In [22]:
model.evaluate(x_test, y_test)

23/23 [==============================] - 3s 80ms/step - loss: 0.3251 - accuracy: 0.9002


[0.32506418228149414, 0.90019291639328]

In [ ]:
# O태그를 많이 예측해서

In [23]:
# F1 score
# f1 = 2*(정밀도*재현율 / 정밀도+재현율)
# 정밀도 : 결과값이 일정하게 분포되어있는지
# 재현율 : 실제 정답인 것 중 모델이 정답이라 예측한 것의 비율

In [24]:
#시퀀스를 NER태그로 변환

def squence_to_tag(sequence):
    result = []
    for seq in sequence:
        temp = []
        for pred in seq:
            # argmax -> 확률이 가장 높은 클래스의 인덱스
            pred_index = np.argmax(pred)
            temp.append(index_to_ner[pred_index].replace('PAD', 'O'))
        result.append(temp)
    return result

In [46]:
y_pred = model.predict(x_test)
pred_tag = squence_to_tag(y_pred)
test_tag = squence_to_tag(y_test)

23/23 [==============================] - 1s 48ms/step


In [56]:
model.evaluate(x_test, y_pred)

23/23 [==============================] - 1s 46ms/step - loss: 0.3107 - accuracy: 1.0000


[0.3106871545314789, 1.0]

In [40]:
word_to_index = sent_tokenizer.word_index
sentence = '삼성전자 출시 스마트폰 오늘 애플 도전장 내밀다.'.split()

new_x = []
for w in sentence:
    try :
        new_x.append(word_to_index.get(w, 1))
    except :
        new_x.append(word_to_index['OOV'])

In [41]:
pad_seq = keras.utils.pad_sequences([new_x], padding='post', value=0, maxlen=40)
p = model.predict(np.array([pad_seq[0]]))
p = np.argmax(p, axis=-1)
p

1/1 [==============================] - 0s 39ms/step


array([[2, 1, 1, 5, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
      dtype=int64)

In [42]:
for w, pred in zip(sentence, p[0]) :
    print('{:10} {:5}'.format(w, index_to_ner[pred]))

삼성전자       I    
출시         O    
스마트폰       O    
오늘         B_DT 
애플         I    
도전장        I    
내밀다.       I    


In [ ]:
# 삼성전자 -> B_OG
# 애플 -> B_OG

In [ ]:
# 오늘 애플 도전장 내밀다
# B_DT  I    I

# 이어지는 데이터가 아닌데 이어진다고 나온 이유 -> 훈련 데이터 부족

In [44]:
# 토큰화 -> 단어 임베딩(원핫인코딩) -> 신경망(합성곱, 순환, 둘다) 구축 -> 모델 통과
# -> NER(BIO)태그 분석 사용자의 의도를 파악 -> 결과를 리턴

# DB연동, 전체 프로세스 구축 API